In [ ]:
import pandas   as pd
import polars   as pl
import numpy    as np
import networkx as nx
import random
import rtsvg
rt      = rtsvg.RACETrack()
relates = [('fm','to')]

# Read in the simple graph templates
df      = pl.read_csv('../rtsvg/config/simple_graphs_df.csv')
g       = rt.createNetworkXGraph(df, relates)

# Create the component lookup structure
comp_lu = {}
for _node_set_ in nx.connected_components(g):
    _g_              = g.subgraph(_node_set_)
    _nodes_, _edges_ = _g_.number_of_nodes(), _g_.number_of_edges()
    if _nodes_ not in comp_lu:          comp_lu[_nodes_]          = {}
    if _edges_ not in comp_lu[_nodes_]: comp_lu[_nodes_][_edges_] = []
    comp_lu[_nodes_][_edges_].append(_g_)

# Positional Information
_df_    = pd.read_csv('../rtsvg/config/simple_graph_layouts.csv')
pos_templates  = {}
for i in range(len(_df_)): pos_templates[_df_['node'][i]] = [_df_['x'][i], _df_['y'][i]]

# Sanity Check
set(df['fm']) | set(df['to']) == set(pos_templates.keys())

In [2]:
#pos = rt.treeMapGraphComponentPlacement(g, pos_templates, bounds_percent=0.3)
#rt.link(df, relates, pos, w=1200,h=900)

In [ ]:
def graphRemoveAllOneDegreeNodes(self, _g_):
    to_remove, removed_nodes = [], {}
    for _node_ in _g_.nodes():
        if _g_.degree(_node_) == 1:
            to_remove.append(_node_)
            _still_in_ = list(_g_.neighbors(_node_))[0]
            if _still_in_ not in removed_nodes: removed_nodes[_still_in_] = set()
            removed_nodes[_still_in_].add(_node_)
    g_after_removal = _g_.copy()
    g_after_removal.remove_nodes_from(to_remove)
    return g_after_removal, removed_nodes

def layoutSimpleTemplates(self, g, pos):
    for _node_set_ in nx.connected_components(g):
        _g_              = g.subgraph(_node_set_)
        _nodes_, _edges_ = _g_.number_of_nodes(), _g_.number_of_edges()
        match_found      = False
        if _nodes_ in comp_lu and _edges_ in comp_lu[_nodes_]:
            for _g_template_ in comp_lu[_nodes_][_edges_]:
                if nx.is_isomorphic(_g_, _g_template_):
                    # If pattern matches, copy the template over
                    gm     = nx.isomorphism.GraphMatcher(_g_, _g_template_)
                    _dict_ = next(gm.subgraph_isomorphisms_iter())
                    for k in _dict_.keys(): pos[k] = pos_templates[_dict_[k]]
                    match_found = True
                    break
        # if no match was found, try the pattern matching with one degree nodes removed
        if not match_found:
            g_after_removal, removed_nodes = graphRemoveAllOneDegreeNodes(self, _g_)
            _nodes_, _edges_ = g_after_removal.number_of_nodes(), g_after_removal.number_of_edges()
            if _nodes_ in comp_lu and _edges_ in comp_lu[_nodes_]:
                for _g_template_ in comp_lu[_nodes_][_edges_]:
                    if nx.is_isomorphic(g_after_removal, _g_template_):
                        # If pattern matches, copy the template over
                        gm     = nx.isomorphism.GraphMatcher(g_after_removal, _g_template_)
                        _dict_ = next(gm.subgraph_isomorphisms_iter())
                        for k in _dict_.keys(): pos[k] = pos_templates[_dict_[k]]
                        # Add the one degrees back in
                        for _node_ in removed_nodes.keys():
                            min_distance_to_nbor = float('inf')
                            for _nbor_ in g_after_removal.neighbors(_node_):
                                d = self.segmentLength((pos[_node_], pos[_nbor_]))
                                if d < min_distance_to_nbor: min_distance_to_nbor = d
                            for _removed_ in removed_nodes[_node_]:
                                pos[_removed_] = (pos[_node_][0] + min_distance_to_nbor/4.0, 
                                                  pos[_node_][1] + min_distance_to_nbor/4.0)
                        match_found = True
                        break

    return self.treeMapGraphComponentPlacement(g, pos, bounds_percent=0.3)

_lu_ = {'src':['a','b','c','d','e','f', 'w', 'x', 'y'],
        'dst':['b','c','a','e','f','d', 'x', 'y', 'z']}
for i in range(random.randint(1,2)):
    if random.random() > 0.5: _lu_['src'].append(f'a{i}'), _lu_['dst'].append(f'a')
    else:                     _lu_['dst'].append(f'a{i}'), _lu_['src'].append(f'a')
for i in range(random.randint(2,3)):
    if random.random() > 0.5: _lu_['src'].append(f'b{i}'), _lu_['dst'].append(f'b')
    else:                     _lu_['dst'].append(f'b{i}'), _lu_['src'].append(f'b')
for i in range(random.randint(3,4)):
    if random.random() > 0.5: _lu_['src'].append(f'c{i}'), _lu_['dst'].append(f'c')
    else:                     _lu_['dst'].append(f'c{i}'), _lu_['src'].append(f'c')
df2   = pl.DataFrame(_lu_)
g_df2 = rt.createNetworkXGraph(df2, [('src','dst')])
pos   = {}
for _node_ in g_df2.nodes(): pos[_node_] = (random.random(), random.random())
pos   = layoutSimpleTemplates(rt, g_df2, pos)
rt.link(df2, [('src','dst')], pos)